In [1]:
import mocet
import os
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import sys
sys.path.append('/DATA/publish/mocet/analysis/scripts')
from utils.base import get_minecraft_subjects, get_project_directory, get_configs
subject_pool = get_minecraft_subjects()
project_dir = get_project_directory()
configs = get_configs()

from numpy.polynomial.legendre import Legendre
from sklearn.linear_model import LinearRegression

def make_poly_regressors(n_samples, order=2):
    X = np.ones((n_samples, 0))
    for d in range(order):
        poly = Legendre.basis(d + 1)
        poly_trend = poly(np.linspace(-1, 1, n_samples))
        X = np.hstack((X, poly_trend[:, None]))
    return X

def polynomial_detrending(pupil_data, polynomial_order):
    X = make_poly_regressors(len(pupil_data), order=polynomial_order)
    dedrift_regressor = np.zeros((len(pupil_data), 2))
    for i in range(2):
        reg = LinearRegression(fit_intercept=False).fit(X, pupil_data[:, i])
        dedrift_regressor[:, i] = reg.predict(X)
    pupil_data = pupil_data[:, :2] - dedrift_regressor
    return pupil_data

calibration_onsets = configs['calibration_onsets']
calibration_points = configs['calibration_points']
interval = configs['interval']
task_duration = configs['task_duration']
task = configs['task']

calibration_offset_start = configs['calibration_offset_start']
calibration_offset_end = configs['calibration_offset_end']
calibration_threshold = configs['calibration_threshold']
px_per_deg = configs['px_per_deg']
avg_pupil_diameter_mm = configs['avg_pupil_diameter_mm']
calibration_coordinates = configs['calibration_coordinates']
calibration_order = configs['calibration_order']

usable_data = pickle.load(open('../data/usable_data_list.pkl', 'rb'))

subjects = []
subjects_runs = {}
for key in list(usable_data.keys()):
    subjects.append(key[0])
    if not subjects_runs.keys().__contains__(key[0]):
        subjects_runs[key[0]] = 0
    subjects_runs[key[0]] += 1
subjects = list(set(subjects))
subjects.sort()

print(len(subjects), subjects)
print("average number of runs:", np.mean(list(subjects_runs.values())),
      "+-", np.std(list(subjects_runs.values())))

18 ['sub-003', 'sub-004', 'sub-005', 'sub-006', 'sub-008', 'sub-009', 'sub-010', 'sub-011', 'sub-015', 'sub-016', 'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 'sub-023', 'sub-024', 'sub-PBJ']
average number of runs: 9.88888888888889 +- 1.8525924445036743


In [2]:
correction_types = ['mocet']
#correction_types = ['mocet', 'polynomial', 'linear', 'uncorrected']
for correction in correction_types:
    for subject in subjects:
        sessions = subject_pool[subject].keys()
        for session in sessions:
            runs = subject_pool[subject][session]
            root = f'{project_dir}/data/eyetracking/{subject}/{session}'
            output_dir = f'../data/corrected_eyetracking/{correction}/{subject}/{session}'
            if not os.path.exists(output_dir):
                os.makedirs(output_dir, exist_ok=True)
            for r in runs:
                run = f'run-{r}'
                np.random.seed(0)
                key = (subject, session, task, run)
                print(correction, key)
                if key in usable_data.keys():
                    # and
                    #     not os.path.exists(f'{output_dir}/{subject}_{session}_{task}_{run}_gaze_coordinate.npy')):
                    log_fname = f'{root}/{subject}_{session}_{task}_{run}_recording-eyetracking_physio_log.csv'
                    data_fname = f'{root}/{subject}_{session}_{task}_{run}_recording-eyetracking_physio_dat.txt'
                    confounds_fname = f'{root}/{subject}_{session}_{task}_{run}_desc-confounds_timeseries.tsv'
                    history_fname = f'{root}/{subject}_{session}_{task}_{run}_recording-eyetracking_physio_his.txt'
                    start, _, _ = mocet.utils.get_viewpoint_history(history_fname)

                    # log, data, confound, start
                    pupil_data, pupil_timestamps, pupil_confidence, _ = mocet.utils.clean_viewpoint_data(log_fname,
                                                                                                         data_fname,
                                                                                                         start=start,
                                                                                                         duration=task_duration)
                    if correction == 'mocet':
                        pupil_data = mocet.apply_mocet(pupil_data=pupil_data,
                                                       motion_params_fname=confounds_fname,
                                                       pupil_confidence=pupil_confidence,
                                                       large_motion_params=False,
                                                       polynomial_order=3)
                    elif correction == 'polynomial':
                        pupil_data = polynomial_detrending(pupil_data, polynomial_order=3)
                    elif correction == 'linear':
                        pupil_data = polynomial_detrending(pupil_data, polynomial_order=1)
                    else:
                        pass

                    offset = calibration_onsets[0]
                    calibration_pupils = []
                    for i in np.arange(calibration_points[0]):
                        start = (offset + i) * interval + calibration_offset_start
                        end = (offset + i + 1) * interval + calibration_offset_end
                        log_effective = np.logical_and(pupil_timestamps >= start * 1000, pupil_timestamps < end * 1000)
                        calibration_pupils.append([np.nanmean(pupil_data[log_effective, 0]),
                                                   np.nanmean(pupil_data[log_effective, 1])])
                    calibration_pupils = np.array(calibration_pupils)

                    calibrator = mocet.EyetrackingCalibration(calibration_coordinates=calibration_coordinates,
                                                              calibration_order=calibration_order,
                                                              repeat=True)
                    calibrator.fit(calibration_pupils[:, 0], calibration_pupils[:, 1])
                    gaze_coordinates = calibrator.transform(pupil_data)

                    np.save(f'{output_dir}/{subject}_{session}_{task}_{run}_gaze_coordinate.npy', gaze_coordinates)
                    np.save(f'{output_dir}/{subject}_{session}_{task}_{run}_gaze_timestamp.npy', pupil_timestamps)

mocet ('sub-003', 'ses-07R', 'task-mcHERDING', 'run-1')
mocet ('sub-003', 'ses-07R', 'task-mcHERDING', 'run-2')
mocet ('sub-003', 'ses-07R', 'task-mcHERDING', 'run-3')
mocet ('sub-003', 'ses-07R', 'task-mcHERDING', 'run-4')
mocet ('sub-003', 'ses-07R', 'task-mcHERDING', 'run-5')
mocet ('sub-003', 'ses-13R', 'task-mcHERDING', 'run-1')
mocet ('sub-003', 'ses-13R', 'task-mcHERDING', 'run-2')
mocet ('sub-003', 'ses-13R', 'task-mcHERDING', 'run-3')
mocet ('sub-003', 'ses-13R', 'task-mcHERDING', 'run-4')
mocet ('sub-003', 'ses-13R', 'task-mcHERDING', 'run-5')
mocet ('sub-003', 'ses-13R', 'task-mcHERDING', 'run-6')
mocet ('sub-003', 'ses-13R', 'task-mcHERDING', 'run-7')
mocet ('sub-004', 'ses-07R', 'task-mcHERDING', 'run-1')
mocet ('sub-004', 'ses-07R', 'task-mcHERDING', 'run-2')
mocet ('sub-004', 'ses-07R', 'task-mcHERDING', 'run-3')
mocet ('sub-004', 'ses-07R', 'task-mcHERDING', 'run-4')
mocet ('sub-004', 'ses-07R', 'task-mcHERDING', 'run-5')
mocet ('sub-004', 'ses-07R', 'task-mcHERDING', '